IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M3\FAULT_M3_5


In [3]:
guasto=1

VOLO m3 FAULT 5%

In [5]:
rcou_m3_fault5 = pd.read_csv("RCOU.csv")
rcou_m3_fault5 = rcou_m3_fault5.astype("float64")
rcou_m3_fault5 = rcou_m3_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m3_fault5 = rcou_m3_fault5[((rcou_m3_fault5['C9'] >= potenza) & (rcou_m3_fault5['C10'] >= potenza) & (rcou_m3_fault5['C11'] >= potenza) & (rcou_m3_fault5['C12'] >= potenza) & (rcou_m3_fault5['C13'] >= potenza) & (rcou_m3_fault5['C14']>= potenza))]
rcou_m3_fault5=rcou_m3_fault5.reset_index(drop=True)
display(rcou_m3_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,204202091.0,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0
1,204212099.0,1369.0,1459.0,1408.0,1424.0,1358.0,1468.0
2,204222805.0,1422.0,1454.0,1486.0,1386.0,1470.0,1405.0
3,204233384.0,1471.0,1448.0,1448.0,1471.0,1457.0,1462.0
4,204243252.0,1509.0,1450.0,1435.0,1522.0,1463.0,1497.0
...,...,...,...,...,...,...,...
8582,303987906.0,1536.0,1428.0,1542.0,1421.0,1443.0,1524.0
8583,303999574.0,1533.0,1428.0,1529.0,1433.0,1427.0,1534.0
8584,304010535.0,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0
8585,304021075.0,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0


In [6]:
min=rcou_m3_fault5['TimeUS'][0]
max=rcou_m3_fault5['TimeUS'][len(rcou_m3_fault5)-1]
print(max)
print(min)

304031184.0
204202091.0


In [7]:
xkf1_m3_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m3_fault5 = xkf1_m3_fault5.astype("float64")
xkf1_m3_fault5 = xkf1_m3_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_fault5 = xkf1_m3_fault5[((xkf1_m3_fault5['TimeUS'] >= min) & (xkf1_m3_fault5['TimeUS'] <= max))]
xkf1_m3_fault5 = xkf1_m3_fault5.reset_index(drop=True)
print(xkf1_m3_fault5)

           TimeUS  Roll  Pitch   Yaw
0     204210668.0  0.88  -0.77  2.52
1     204220693.0  0.85  -0.83  2.51
2     204230930.0  0.91  -0.79  2.53
3     204241781.0  0.94  -0.72  2.56
4     204251811.0  1.02  -0.76  2.56
...           ...   ...    ...   ...
9085  303985816.0  1.10  -0.30  2.01
9086  303997185.0  1.06  -0.25  2.00
9087  304007782.0  0.97  -0.22  1.97
9088  304018840.0  0.93  -0.14  1.94
9089  304029034.0  0.96  -0.06  1.90

[9090 rows x 4 columns]


In [8]:
att_m3_fault5 = pd.read_csv("ATT.csv")
att_m3_fault5 = att_m3_fault5.astype("float64")
att_m3_fault5 = att_m3_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m3_fault5 = att_m3_fault5[((att_m3_fault5['TimeUS'] >= min) & (att_m3_fault5['TimeUS'] <= max	))]
att_m3_fault5=att_m3_fault5.reset_index(drop=True)
print(att_m3_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0     204211309.0     0.00  0.88      0.00  -0.77    2.49  2.52
1     204222059.0     0.00  0.85      0.00  -0.83    2.49  2.51
2     204232678.0     0.00  0.91      0.00  -0.79    2.49  2.53
3     204242578.0     0.00  0.94      0.00  -0.72    2.49  2.56
4     204253192.0     0.00  1.02      0.00  -0.76    2.49  2.56
...           ...      ...   ...       ...    ...     ...   ...
9086  303987111.0     2.69  1.10      1.47  -0.30    1.71  2.01
9087  303998743.0     2.66  1.06      1.49  -0.25    1.71  2.00
9088  304009828.0     2.65  0.97      1.52  -0.22    1.71  1.97
9089  304020291.0     2.64  0.93      1.55  -0.14    1.71  1.94
9090  304030671.0     2.63  0.96      1.58  -0.06    1.71  1.90

[9091 rows x 7 columns]


In [9]:
from numpy import empty

if len(att_m3_fault5)>len(xkf1_m3_fault5):
    to_add=att_m3_fault5[len(xkf1_m3_fault5):]
    att_m3_fault5=att_m3_fault5[:len(xkf1_m3_fault5)]

if len(xkf1_m3_fault5)>len(att_m3_fault5):
    to_add=xkf1_m3_fault5[len(att_m3_fault5):]

for idx,i in enumerate(att_m3_fault5['Roll']):
    if(xkf1_m3_fault5['Roll'][idx] != i):
        att_m3_fault5['Roll'][idx] = ((att_m3_fault5['Roll'][idx] + xkf1_m3_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m3_fault5['Pitch']):
    if(xkf1_m3_fault5['Pitch'][idx] != i):
        att_m3_fault5['Pitch'][idx] = ((att_m3_fault5['Pitch'][idx] + xkf1_m3_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m3_fault5['Yaw']):
    if(xkf1_m3_fault5['Yaw'][idx] != i):
        att_m3_fault5['Yaw'][idx] = ((att_m3_fault5['Yaw'][idx] + xkf1_m3_fault5['Yaw'][idx])/2)

if not to_add.empty:
    att_m3_fault5=pd.concat([att_m3_fault5,to_add])

print(att_m3_fault5)

           TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw
0     204211309.0     0.00  0.880      0.00 -0.770    2.49  2.520
1     204222059.0     0.00  0.850      0.00 -0.830    2.49  2.510
2     204232678.0     0.00  0.910      0.00 -0.790    2.49  2.530
3     204242578.0     0.00  0.940      0.00 -0.720    2.49  2.560
4     204253192.0     0.00  1.020      0.00 -0.760    2.49  2.560
...           ...      ...    ...       ...    ...     ...    ...
9086  303987111.0     2.69  1.080      1.47 -0.275    1.71  2.005
9087  303998743.0     2.66  1.015      1.49 -0.235    1.71  1.985
9088  304009828.0     2.65  0.950      1.52 -0.180    1.71  1.955
9089  304020291.0     2.64  0.945      1.55 -0.100    1.71  1.920
9090  304030671.0     2.63  0.960      1.58 -0.060    1.71  1.900

[9091 rows x 7 columns]


In [10]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m3_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m3_fault5 = esc_0_m3_fault5.astype("float64")
esc_0_m3_fault5 = esc_0_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m3_fault5 = esc_0_m3_fault5[((esc_0_m3_fault5['TimeUS'] >= min) & (esc_0_m3_fault5['TimeUS'] <= max))]
esc_0_m3_fault5=esc_0_m3_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m3_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m3_fault5 = esc_1_m3_fault5.astype("float64")
esc_1_m3_fault5 = esc_1_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m3_fault5 = esc_1_m3_fault5[((esc_1_m3_fault5['TimeUS'] >= min) & (esc_1_m3_fault5['TimeUS'] <= max))]
esc_1_m3_fault5=esc_1_m3_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m3_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m3_fault5 = esc_2_m3_fault5.astype("float64")
esc_2_m3_fault5 = esc_2_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m3_fault5 = esc_2_m3_fault5[((esc_2_m3_fault5['TimeUS'] >= min) & (esc_2_m3_fault5['TimeUS'] <= max))]
esc_2_m3_fault5=esc_2_m3_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m3_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m3_fault5 = esc_3_m3_fault5.astype("float64")
esc_3_m3_fault5 = esc_3_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m3_fault5 = esc_3_m3_fault5[((esc_3_m3_fault5['TimeUS'] >= min) & (esc_3_m3_fault5['TimeUS'] <= max))]
esc_3_m3_fault5=esc_3_m3_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m3_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m3_fault5 = esc_4_m3_fault5.astype("float64")
esc_4_m3_fault5 = esc_4_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m3_fault5 = esc_4_m3_fault5[((esc_4_m3_fault5['TimeUS'] >= min) & (esc_4_m3_fault5['TimeUS'] <= max))]
esc_4_m3_fault5=esc_4_m3_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m3_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m3_fault5 = esc_5_m3_fault5.astype("float64")
esc_5_m3_fault5 = esc_5_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m3_fault5 = esc_5_m3_fault5[((esc_5_m3_fault5['TimeUS'] >= min) & (esc_5_m3_fault5['TimeUS'] <= max))]
esc_5_m3_fault5=esc_5_m3_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [11]:
#valori di accelerazione e del giroscopio
imu_0_m3_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m3_fault5 = imu_0_m3_fault5.astype("float64")
imu_0_m3_fault5 = imu_0_m3_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m3_fault5 = imu_0_m3_fault5[((imu_0_m3_fault5['TimeUS'] >= min) & (imu_0_m3_fault5['TimeUS'] <= max))]

imu_1_m3_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m3_fault5 = imu_1_m3_fault5.astype("float64")
imu_1_m3_fault5 = imu_1_m3_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m3_fault5 = imu_1_m3_fault5[((imu_1_m3_fault5['TimeUS'] >= min) & (imu_1_m3_fault5['TimeUS'] <= max))]

imu_2_m3_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m3_fault5 = imu_2_m3_fault5.astype("float64")
imu_2_m3_fault5 = imu_2_m3_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m3_fault5 = imu_2_m3_fault5[((imu_2_m3_fault5['TimeUS'] >= min) & (imu_2_m3_fault5['TimeUS'] <= max))]

imu_m3_fault5 = pd.concat((imu_0_m3_fault5, imu_1_m3_fault5, imu_2_m3_fault5))
imu_m3_fault5=imu_m3_fault5.groupby(imu_m3_fault5.TimeUS, as_index=False).mean()

display(imu_m3_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,204204632.0,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536
1,204206099.0,0.026523,-0.083041,0.008797,-0.019900,-0.217393,-9.794001
2,204209242.0,-0.143925,0.062323,0.064051,-0.122822,-0.059922,-10.048777
3,204211153.0,-0.172116,0.091631,0.076426,-0.149506,0.052869,-10.198143
4,204213901.0,-0.035754,0.064878,0.045289,-0.319344,0.122338,-10.160297
...,...,...,...,...,...,...,...
34503,304020133.0,-0.043775,0.158520,-0.049635,-0.113556,0.259399,-10.788323
34504,304022922.0,0.050244,0.063406,-0.067958,0.036619,0.395237,-10.919890
34505,304025102.0,0.065195,0.030756,-0.069719,-0.167383,0.277658,-10.851300
34506,304027773.0,-0.008855,0.096466,-0.051342,-0.315915,0.138011,-10.782627


In [12]:
#sincronizzazione dei tempi tra att e imu_2
m3_fault5 = pd.merge_ordered(imu_m3_fault5,att_m3_fault5)
m3_fault5=m3_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_0_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_1_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_2_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_3_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_4_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_5_m3_fault5)
m3_fault5=m3_fault5.fillna(method="ffill").fillna(method="bfill")
m3_fault5 = pd.merge_ordered(m3_fault5, rcou_m3_fault5)
m3_fault5=m3_fault5.fillna(method="ffill").fillna(method="bfill")

In [13]:
m3_fault5["TimeUS"] = m3_fault5["TimeUS"] - m3_fault5.iloc[0]["TimeUS"]
m3_fault5["TimeUS"] = m3_fault5["TimeUS"].astype(int)
m3_fault5["Guasto"] = guasto

In [14]:
last_cell = m3_fault5.iloc[-1, m3_fault5.columns.get_loc('TimeUS')]
print(last_cell)

99829093


In [15]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [16]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
34937  99815009
34938  99817866
34939  99820723
34940  99823580
34941  99826437

[34942 rows x 1 columns]


In [17]:
m3_fault5_final = pd.merge_ordered(m3_fault5,df)
m3_fault5_final=m3_fault5_final.fillna(method="ffill").fillna(method="bfill")
print(m3_fault5_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.115041 -0.156477 -0.036312  0.172309 -0.134253  -9.789536   
1          2541  0.115041 -0.156477 -0.036312  0.172309 -0.134253  -9.789536   
2          2712  0.115041 -0.156477 -0.036312  0.172309 -0.134253  -9.789536   
3          2857  0.115041 -0.156477 -0.036312  0.172309 -0.134253  -9.789536   
4          4008  0.026523 -0.083041  0.008797 -0.019900 -0.217393  -9.794001   
...         ...       ...       ...       ...       ...       ...        ...   
92192  99825682 -0.008855  0.096466 -0.051342 -0.315915  0.138011 -10.782627   
92193  99826437 -0.008855  0.096466 -0.051342 -0.315915  0.138011 -10.782627   
92194  99828166 -0.052177  0.183527 -0.037859 -0.200246  0.199173 -10.854337   
92195  99828580 -0.052177  0.183527 -0.037859 -0.200246  0.199173 -10.854337   
92196  99829093 -0.052177  0.183527 -0.037859 -0.200246  0.199173 -10.854337   

       DesRoll   Roll  DesPitch  ...  C

In [18]:
m3_fault5_final = m3_fault5_final[m3_fault5_final.TimeUS.isin(to_be)]

In [19]:
m3_fault5_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
3,2857,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
5,5714,0.026523,-0.083041,0.008797,-0.019900,-0.217393,-9.794001,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
7,8571,-0.143925,0.062323,0.064051,-0.122822,-0.059922,-10.048777,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
11,11428,-0.172116,0.091631,0.076426,-0.149506,0.052869,-10.198143,0.00,0.880,0.00,...,2.74,1781.0,3.11,1369.0,1459.0,1408.0,1424.0,1358.0,1468.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92183,99815009,-0.041717,0.132493,-0.056576,-0.175003,0.174827,-10.644793,2.65,0.950,1.52,...,1.96,5735.0,3.18,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0,1.0
92184,99817866,-0.041717,0.132493,-0.056576,-0.175003,0.174827,-10.644793,2.65,0.950,1.52,...,1.96,5735.0,3.18,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0,1.0
92188,99820723,-0.043775,0.158520,-0.049635,-0.113556,0.259399,-10.788323,2.64,0.945,1.55,...,1.96,5735.0,3.18,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1.0
92191,99823580,0.065195,0.030756,-0.069719,-0.167383,0.277658,-10.851300,2.64,0.945,1.55,...,1.96,5735.0,3.18,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1.0


In [20]:
from datetime import timedelta
m3_fault5_final=m3_fault5_final.reset_index(drop=True)
m3_fault5_final['TimeUS'] = pd.to_datetime(m3_fault5_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m3_fault5_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
1,00:00:00.002857,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
2,00:00:00.005714,0.026523,-0.083041,0.008797,-0.019900,-0.217393,-9.794001,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
3,00:00:00.008571,-0.143925,0.062323,0.064051,-0.122822,-0.059922,-10.048777,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
4,00:00:00.011428,-0.172116,0.091631,0.076426,-0.149506,0.052869,-10.198143,0.00,0.880,0.00,...,2.74,1781.0,3.11,1369.0,1459.0,1408.0,1424.0,1358.0,1468.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34937,00:01:39.815009,-0.041717,0.132493,-0.056576,-0.175003,0.174827,-10.644793,2.65,0.950,1.52,...,1.96,5735.0,3.18,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0,1.0
34938,00:01:39.817866,-0.041717,0.132493,-0.056576,-0.175003,0.174827,-10.644793,2.65,0.950,1.52,...,1.96,5735.0,3.18,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0,1.0
34939,00:01:39.820723,-0.043775,0.158520,-0.049635,-0.113556,0.259399,-10.788323,2.64,0.945,1.55,...,1.96,5735.0,3.18,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1.0
34940,00:01:39.823580,0.065195,0.030756,-0.069719,-0.167383,0.277658,-10.851300,2.64,0.945,1.55,...,1.96,5735.0,3.18,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1.0


## Analisi nel tempo e in frequenza

In [21]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [22]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m3_fault5_final)/350)):
        V1=m3_fault5_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [23]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_4364\162550328.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_4364\162550328.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,0.010773,0.007645,-0.477744,0.087972,75.0,62.0,1.783221,1.423987,0.023235,...,13255.936058,4.065470,0.877341,-1.510801,4.171677,128.0,129.0,47.149866,32.182781,1
1,1.0,0.010485,0.007652,-0.481629,0.087979,58.0,74.0,1.881163,1.705376,0.022757,...,13068.713703,4.064091,0.875511,-1.506622,4.170115,131.0,132.0,48.445625,31.556443,1
2,2.0,0.010365,0.007636,-0.476359,0.087871,72.0,89.0,2.114065,1.765686,0.022350,...,12861.204335,4.062720,0.873687,-1.502421,4.168561,126.0,127.0,49.651477,30.873413,1
3,3.0,0.010313,0.007615,-0.469214,0.087748,67.0,76.0,1.863813,1.836036,0.022236,...,12633.546742,4.061356,0.871871,-1.498199,4.167016,131.0,132.0,50.764742,30.134296,1
4,4.0,0.010103,0.007609,-0.468658,0.087691,77.0,74.0,1.835840,1.652635,0.022070,...,12385.911251,4.060000,0.870060,-1.493957,4.165478,140.0,141.0,51.783188,29.339792,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,94.0,-0.003359,0.007144,-0.260725,0.084496,0.0,103.0,5.380491,2.485981,0.012145,...,13134.588325,4.021011,0.757519,-1.305505,4.113921,169.0,170.0,50.722221,45.408406,1
95,95.0,-0.003539,0.007143,-0.259690,0.084493,0.0,95.0,5.372192,2.070122,0.012078,...,13353.735986,4.022489,0.756790,-1.308451,4.115278,169.0,170.0,48.624276,44.308898,1
96,96.0,-0.003921,0.007192,-0.257646,0.084800,0.0,89.0,5.273038,1.745373,0.011894,...,13564.417617,4.023960,0.756061,-1.311343,4.116627,-173.0,-172.0,46.468275,43.102284,1
97,97.0,-0.004383,0.007271,-0.235416,0.085289,0.0,109.0,5.137177,1.685271,0.011759,...,13766.154314,4.025424,0.755330,-1.314182,4.117971,-169.0,-168.0,44.266748,41.791710,1


In [24]:
path_file = path_file.replace(r"M3\FAULT_M3_5", "")
os.chdir(path_file)
df_merged.to_csv('m3_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
